# Basic HF API connection

In [1]:
%pip install -q huggingface-hub==0.23.2
%pip install -q transformers==4.47.0
%pip install -q datasets==2.19.1
%pip install -q sentence-transformers==2.7.0
%pip install -q optuna==3.6.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 5.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.14.0 requires huggingface-hub>=0.25.0, but you have huggingface-hub 0.23.2 which is incompatible.
transformers 4.47.1 requires huggingface-hub<1.0,>=0.24.0, but you have huggingface-hub 0.23.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 0. Setup

In [2]:
import os
import yaml
from google.colab import drive
from getpass import getpass

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Read YAML file
f_path = "/content/drive/MyDrive/GitHub/python-codebase/machine_learning/private_keys.yml"
with open(f_path, 'r') as stream:
    data_loaded = yaml.safe_load(stream)
os.environ['HF_API_TOKEN'] = data_loaded['HF_API_KEY']
os.environ['GITHUB_TOKEN'] = data_loaded['GITHUB_TOKEN']

In [4]:
# Set up token
from huggingface_hub import login
login(token=os.environ['HF_API_TOKEN'])

In [5]:
# Example (directly accesing HF endpoint w/ requests)
#model_name = "mistralai/Mistral-7B-Instruct-v0.3"
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
#model_name = "Qwen/Qwen2.5-72B-Instruct"
API_URL = f"https://api-inference.huggingface.co/models/{model_name}"
headers = {"Authorization": f"Bearer {os.environ['HF_API_TOKEN']}"}

In [6]:
from google.colab import userdata

## 1. Example

In [ ]:
import requests
import numpy as np
from tqdm import tqdm
from datasets import load_dataset
from joblib import Parallel, delayed
from sentence_transformers import SentenceTransformer, util

In [ ]:
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

def get_output(input_prompt, dct_params={'max_new_tokens': 250, 'temperature': 0.1}, mod=True):
  data = query({'inputs': input_prompt, 'parameters': dct_params})
  try:
    data =  data[0]['generated_text']
  except:
    print(data)
    raise ValueError()
  if mod:
    data = data.split(input_prompt)[-1] # Ensure it does not contain the input prompt
  return data

In [ ]:
input_prompt = "Write a SQL query that contains a JOIN and a GROUP BY"
output = get_output(input_prompt, dct_params={'max_new_tokens': 250, 'temperature': 0.1}, mod=False)
print(output)

Write a SQL query that contains a JOIN and a GROUP BY clause to find the number of orders for each customer.

The query should return the customer id, the customer name, and the number of orders.

The tables are:

* Customers: id, name
* Orders: id, customer\_id

```
SELECT Customers.id, Customers.name, COUNT(Orders.id) AS num_orders
FROM Customers
JOIN Orders
ON Customers.id = Orders.customer_id
GROUP BY Customers.id, Customers.name;
```
This query first joins the Customers and Orders tables on the customer\_id column, then groups the results by the customer id and name. The COUNT function is used to count the number of orders for each group.


## 2. Otro ejemplo

In [ ]:
from huggingface_hub import InferenceClient

In [ ]:
model_name = "Qwen/Qwen2.5-72B-Instruct"
#model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"

dct_params={'max_new_tokens': 1000, 'temperature': 0.1, 'return_full_text': False}

In [ ]:
input_prompt = "Write a SQL query that contains a JOIN and a GROUP BY"

client = InferenceClient()
output = client.text_generation(input_prompt, model=model_name, **dct_params)
print(output)

 clause. The query should join two tables, "orders" and "customers", and return the total number of orders and the total amount spent by each customer. Additionally, the query should only include customers who have placed more than 5 orders and spent more than $500 in total. The result should be ordered by the total amount spent in descending order.
Sure! Below is a SQL query that meets your requirements:

```sql
SELECT 
    c.customer_id,
    c.customer_name,
    COUNT(o.order_id) AS total_orders,
    SUM(o.order_amount) AS total_spent
FROM 
    customers c
JOIN 
    orders o ON c.customer_id = o.customer_id
GROUP BY 
    c.customer_id, c.customer_name
HAVING 
    COUNT(o.order_id) > 5 AND SUM(o.order_amount) > 500
ORDER BY 
    total_spent DESC;
```

### Explanation:
1. **SELECT Clause**: 
   - `c.customer_id` and `c.customer_name` are selected to identify the customer.
   - `COUNT(o.order_id)` is used to count the total number of orders placed by each customer.
   - `SUM(o.order_amo

In [ ]:
model_name = "microsoft/Phi-3.5-mini-instruct"

input_prompt = "Write a SQL query that contains a JOIN and a GROUP BY"

client = InferenceClient()
output = client.text_generation(input_prompt, model=model_name, **dct_params)
print(output)

 clause to find the total number of orders placed by each customer in the 'customers' and 'orders' tables.

Table: customers

| id | name   |
|----|--------|
| 1  | Alice  |
| 2  | Bob    |
| 3  | Carol  |

Table: orders

| id | customer_id | date       |
|----|-------------|------------|
| 1  | 1           | 2020-01-01 |
| 2  | 2           | 2020-01-02 |
| 3  | 1           | 2020-01-03 |
| 4  | 3           | 2020-01-04 |
| 5  | 2           | 2020-01-05 |

The SQL query is:

```sql
SELECT c.name, COUNT(o.id) AS total_orders
FROM customers c
JOIN orders o ON c.id = o.customer_id
GROUP BY c.name;
```

This query joins the 'customers' and 'orders' tables on the 'id' and 'customer_id' columns, respectively. It then groups the results by the customer name and counts the number of orders for each customer using the COUNT() function. The result is a table with two columns: 'name' and 'total_orders', where each row represents a customer and their total number of orders.

The output of the quer

## 3. Cargar modelos en memoria

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
## Load model and set up tokenizer
# Bloom: Bloom is one of the smallest and smartest models available for training with the PEFT Library using Prompt Tuning

model_name = "bigscience/bloomz-560m"
# model_name="bigscience/bloom-1b1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
foundational_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True
)

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [8]:
## Inference with the pre trained bloom model
# this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs):
    dct_params={'max_new_tokens': 1000, 'temperature': 0.1}
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        repetition_penalty=1.5,  # Avoid repetition.
        early_stopping=True,  # The model can stop before reach the max_length
        eos_token_id=tokenizer.eos_token_id,
        **dct_params
    )
    return outputs

In [13]:
input_prompt = "Write a SQL query that contains a JOIN and a GROUP BY"

input_sentences = tokenizer(
    input_prompt,
    return_tensors="pt"
)
foundational_outputs_sentence = get_outputs(
    foundational_model,
    input_sentences
)
print(tokenizer.batch_decode(foundational_outputs_sentence, skip_special_tokens=True))

['Write a SQL query that contains a JOIN and a GROUP BY clause. \nSELECT * FROM table WHERE (group_or_like(name, \'US\') LIKE \'%" + group_name() %"\' OR name = " - 1" ) ORDER by id DESC LIMIT 1']


### 3.1. Versión mejorada

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [10]:
model_name = "bigscience/bloomz-560m"
# model_name="bigscience/bloom-1b1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
foundational_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True
)

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [11]:
def llm_call(tokenizer: object, foundational_model:object, prompt: str, system_prompt: str = "") -> str:
    """
    Calls the model with the given prompt and returns the response.

    NOTE: USes a preloaded HF Model

    Args:
        prompt (str): The user prompt to send to the model.
        system_prompt (str, optional): The system prompt to send to the model. Defaults to "".

    Returns:
        str: The response from the language model.
    """
    if system_prompt != "":
      input_prompt = system_prompt + '\n\n' + prompt
    else:
      input_prompt = prompt

    input_sentences = tokenizer(
        input_prompt,
        return_tensors="pt"
    )
    foundational_outputs_sentence = get_outputs(
        foundational_model,
        input_sentences
    )
    response = tokenizer.batch_decode(
        foundational_outputs_sentence,
        skip_special_tokens=True
        )
    response = response[0]
    response = response.split(input_prompt)[-1]

    return response

In [12]:
input_prompt = "Write a SQL query that contains a JOIN and a GROUP BY"
output = llm_call(
    tokenizer=tokenizer,
    foundational_model=foundational_model,
    prompt=input_prompt,
    system_prompt=""
    )
print(output)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


 clause. 
SELECT * FROM table WHERE (group_or_like(name, 'US') LIKE '%" + group_name() %"' OR name = " - 1" ) ORDER by id DESC LIMIT 1


### 3.2. Otros modelos

In [14]:
# model_name = "bigscience/bloomz-560m"
# model_name="bigscience/bloom-1b1"
# model_name = "microsoft/Phi-3.5-mini-instruct"
model_name = "MiniLLM/MiniPLM-Qwen-500M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
foundational_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True
)

tokenizer_config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.86G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [17]:
input_prompt = "Write a SQL query that contains a JOIN and a GROUP BY"
output = llm_call(
    tokenizer=tokenizer,
    foundational_model=foundational_model,
    prompt=input_prompt,
    system_prompt=""
    )
print(output)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


 clause.  The join should be the same as your original SELECT statement, but you can use an alias to make it more readable.
SELECT
    t1.id,
FROM table2 AS T3 
JOIN (
   -- Join on id = (id from TableA)
      LEFT OUTER JOIN TABLEB ON ID IN ((TableC.ID),(TABLED.FK))
) A USING(id)


